# Support Vector Machine Learning Model (96% variance - F1 Macro)

Import required packages

In [1]:
import numpy as np # for multi-dimensional array operations
import pandas as pd # for reading data from .csv files
from sklearn.svm import SVC # for support vector machine model
from sklearn.decomposition import PCA # for principle component analysis (dimensionality reduction)
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold # for getting the best hyper parameters

Assign the training set and testing set to variables for easy reference

In [2]:
train_set = pd.read_csv('../train_tfidf_features.csv') # import the training set
test_set = pd.read_csv('../test_tfidf_features.csv') # import the testing set

Principal Component Analysis for train_set (96% variance)

In [3]:
train_set_label = train_set.loc[:, ["label"]]
features_names = [str(i) for i in range(0, 5000)]
train_set_features = train_set.loc[:, features_names] # train_set_features will not contain the label and id columns

# perform PCA
pca = PCA(n_components = 0.96)
train_set_reduced = pca.fit_transform(train_set_features)
train_set_reduced = pd.DataFrame(data = train_set_reduced)
train_set_reduced

,0,1,2,3,4,5,6,7,8,9,...,3624,3625,3626,3627,3628,3629,3630,3631,3632,3633
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,0.019296,0.011578,0.004796,0.002857,0.000321,-0.008228,0.006793,-0.006243,0.002063,0.003779
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.003708,-0.002765,0.001527,0.002005,-0.003894,-0.004366,0.004768,-0.006414,-0.000668,-0.002256
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,-0.006987,0.003821,0.010419,-0.004462,0.000807,0.009793,0.006161,-0.007252,0.004770,-0.003063
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.006774,-0.000069,-0.001450,0.000938,0.007665,0.006279,0.007776,0.000753,-0.001768,0.012034
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.014709,0.010219,-0.008368,-0.008664,-0.003762,-0.007937,0.004112,-0.015324,0.009286,0.007978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,-0.001357,-0.002034,0.002664,-0.001143,0.001539,0.001269,0.011205,-0.002811,0.002754,-0.005535
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,0.004056,0.007848,0.008196,-0.009477,0.008709,-0.001505,0.013458,0.000002,0.009011,0.008338
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,-0.000709,-0.015198,0.001911,0.001970,-0.003440,-0.006773,0.011027,0.006825,0.005706,-0.014734
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.003035,0.002357,-0.002707,0.002088,-0.006140,0.002185,0.001989,-0.000054,0.000885,0.000121


In [4]:
X = train_set_reduced
y = train_set_label

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) # Train a SVC model using different kernel
X_train = X
y_train = y

In [5]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,3624,3625,3626,3627,3628,3629,3630,3631,3632,3633
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,0.019296,0.011578,0.004796,0.002857,0.000321,-0.008228,0.006793,-0.006243,0.002063,0.003779
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.003708,-0.002765,0.001527,0.002005,-0.003894,-0.004366,0.004768,-0.006414,-0.000668,-0.002256
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,-0.006987,0.003821,0.010419,-0.004462,0.000807,0.009793,0.006161,-0.007252,0.004770,-0.003063
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,-0.006774,-0.000069,-0.001450,0.000938,0.007665,0.006279,0.007776,0.000753,-0.001768,0.012034
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.014709,0.010219,-0.008368,-0.008664,-0.003762,-0.007937,0.004112,-0.015324,0.009286,0.007978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,-0.001357,-0.002034,0.002664,-0.001143,0.001539,0.001269,0.011205,-0.002811,0.002754,-0.005535
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,0.004056,0.007848,0.008196,-0.009477,0.008709,-0.001505,0.013458,0.000002,0.009011,0.008338
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,-0.000709,-0.015198,0.001911,0.001970,-0.003440,-0.006773,0.011027,0.006825,0.005706,-0.014734
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.003035,0.002357,-0.002707,0.002088,-0.006140,0.002185,0.001989,-0.000054,0.000885,0.000121


In [6]:
y_train

,label
0,1
1,0
2,1
3,0
4,1
...,...
17179,0
17180,0
17181,1
17182,1


Principal Component Analysis for test_set (96% variance)

In [7]:
features_names = [str(i) for i in range(0, 5000)]
test_set_features = test_set.loc[:, features_names] # test_set_features will not contain the label and id columns

# perform PCA
test_set_reduced = pca.transform(test_set_features) # use the pca from the train_set?
test_set_features = pd.DataFrame(data = test_set_reduced)
test_set_features

,0,1,2,3,4,5,6,7,8,9,...,3624,3625,3626,3627,3628,3629,3630,3631,3632,3633
0,-0.082801,-0.009343,-0.013543,-0.013317,-0.007984,-0.002459,-0.026278,0.004096,-0.014072,-0.020606,...,-0.005453,-0.001511,0.004597,0.000499,0.002554,0.001095,-0.003870,-0.002089,0.004334,-0.006367
1,-0.075595,0.005040,-0.017771,-0.014935,-0.000467,-0.001601,-0.055948,-0.017461,-0.031185,-0.017278,...,-0.003447,-0.007628,-0.009233,-0.001764,-0.005944,-0.001453,-0.005794,-0.002982,-0.000716,0.001860
2,0.163876,0.059162,-0.057706,-0.040884,-0.063164,-0.003223,0.020473,-0.005967,-0.005539,-0.008680,...,-0.006349,-0.006289,0.006995,-0.004119,-0.009040,0.007147,-0.005139,-0.002119,0.002720,-0.008454
3,-0.006835,0.189357,-0.044389,-0.024452,-0.043835,-0.013077,0.001344,0.000674,-0.005021,-0.028871,...,-0.004599,-0.001363,0.002356,-0.003300,-0.003127,-0.001100,-0.004519,0.000457,-0.001947,-0.001643
4,0.120704,-0.089485,0.015896,0.030057,-0.018015,-0.057111,-0.072387,0.053490,-0.153091,0.026020,...,0.000264,0.003480,-0.003061,-0.000700,-0.002121,-0.003195,-0.002934,-0.001807,0.001749,-0.003604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,-0.076808,-0.024913,-0.014752,-0.008106,-0.009817,-0.013137,0.026288,-0.012828,-0.004845,-0.010727,...,-0.002452,0.004073,-0.000257,-0.000386,0.001614,-0.002238,0.006151,0.000733,0.006419,0.004016
4292,0.069331,-0.055078,-0.008249,-0.022183,-0.000882,0.024624,-0.085988,-0.152147,0.159634,0.018128,...,-0.007111,0.005197,0.007090,-0.005912,0.003421,0.006620,0.004157,-0.002099,-0.005733,-0.002524
4293,0.042192,-0.056123,-0.009015,-0.022608,-0.000348,0.011589,-0.046442,-0.129357,0.126640,0.010080,...,0.004017,0.003642,0.005206,-0.006316,0.005551,0.005955,-0.005016,-0.000628,0.000189,-0.001986
4294,0.220417,-0.101562,-0.013420,-0.014971,-0.033857,-0.007979,-0.018445,-0.004240,-0.019587,-0.000660,...,0.007371,-0.005735,0.007448,0.011382,0.007971,-0.024975,0.011738,0.006900,0.006600,0.011672


Tuning the hyper-parameters and training the model based on the best hyper-parameters

In [8]:
# hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']} # initialise the hyper-parameters
hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} # initialise the hyper-parameters
kfold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0) # for 3-fold cross validation
# grid = GridSearchCV(SVC(), hyper_parameters, refit = True, verbose = 2) # cretae a GridSearchCV object to git to the taining data
grid = GridSearchCV(SVC(), param_grid = hyper_parameters, scoring = 'f1_macro', refit = 'f1_macro', n_jobs = 1 , cv = kfold, verbose = 2)
grid.fit(X_train, np.ravel(y_train)) # training the model using the best hyper-parameters
print(grid.best_params_) # gets the best hyper-parameters for SVM

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 6.9min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 7.1min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 8.0min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 6.5min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 6.6min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 6.7min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 6.4min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 6.4min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 6.3min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 6.2min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 6.1min
[CV] END .....................C=0.1, gamma=0.001

Predicting the labels for the test dataset based on the model with the best hyper-parameters

In [9]:
y_predicted = grid.predict(test_set_features)

In [11]:
# y_predicted = svc_model.predict(test_set_features)
y_predicted = pd.DataFrame(y_predicted, columns = ['label']) # convert y_predicted from nparray to pandas dataframe
y_predicted.insert(loc = 0, column = 'id', value = [i for i in range(17185, 17185 + 4296)]) # insert a column of the ids, starting from 17185
y_predicted.to_csv('skynet_submission_96_f1macro.csv', index = False) # output the predicted labels to ./skynet_submission_96_f1macro.csv